In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import nltk #language processing
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

import nltk
import pandas as pd
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

import re
import emoji


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gokha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gokha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gokha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gokha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data_train = pd.read_csv("congressional_tweet_training_data.csv",names=None)
data_test= pd.read_csv("congressional_tweet_test_data.csv",names=None)

In [3]:
data_train.head()

,favorite_count,full_text,hashtags,retweet_count,year,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D


In [4]:
data_test.head()

,Id,favorite_count,full_text,hashtags,retweet_count,year,party
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D
1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D
2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D
3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D
4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D


In [5]:
data_train['party_class']=data_train['party_id'].apply(lambda x: 0 if x=='D' else 1)

In [6]:
data_test['party_class']=data_test['party'].apply(lambda x: 0 if x=='D' else 1)

In [7]:
# Start by cleaning the full text of the tweets 
data_train.dropna(subset=['full_text'],inplace=True) #First, I drop the rows with NA in the text column
data_train.head()

,favorite_count,full_text,hashtags,retweet_count,year,party_id,party_class
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,1
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,1
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,1
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,1
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,0


In [8]:
data_train=data_train.reset_index(drop=True) #to fix the indices

In [9]:
#data_train=data_train[0:100]

In [ ]:
# Tokenize and tag full version
#tagged_tokens=[pos_tag(word_tokenize(sent)) for sent in data_train['full_text']]


# apply version
# data_train['tokenized_text'] = data_train['full_text'].apply(word_tokenize) 
# data_train['tagged_tokens'] = data_train['tokenized_text'].apply(pos_tag)


### seperate version


In [13]:
# Tokenize
tokens_list=[word_tokenize(sent) for sent in data_train['full_text']]

In [ ]:
# Pos Tag 
tagged_tokens =[pos_tag(sent) for sent in tokens_list]

In [15]:
def get_wordnet_pos(treebank_tag): #Following from [16]

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None # for easy if-statement 

In [16]:
lemmatizer = WordNetLemmatizer()

In [17]:
x=pd.DataFrame(data_train['full_text'])

In [18]:
x['tagged_tokens']=tagged_tokens

In [19]:
lemma_list = x['tagged_tokens'].transform(lambda value: ' '.join([lemmatizer.lemmatize(a[0],pos=get_wordnet_pos(a[1])) if get_wordnet_pos(a[1]) else a[0] for a in  value]))

In [20]:
x['lemma_list']=lemma_list

In [21]:
x.head()

,full_text,tagged_tokens,lemma_list
0,"b""RT @KUSINews: One of our longtime viewers wa...","[(b, NN), ('', ''), (RT, NNP), (@, NNP), (KUSI...",b '' RT @ KUSINews : One of our longtime viewe...
1,"b""Today I'm urging the @CDCgov to immediately ...","[(b, NN), ('', ''), (Today, NN), (I, PRP), ('m...",b '' Today I 'm urge the @ CDCgov to immediate...
2,"b'Tomorrow, #MO03 seniors graduate from Calvar...","[(b'Tomorrow, NN), (,, ,), (#, #), (MO03, NNP)...","b'Tomorrow , # MO03 senior graduate from Calva..."
3,b'Congrats to #TeamUSA and Canton Native @JGre...,"[(b'Congrats, NNS), (to, TO), (#, #), (TeamUSA...",b'Congrats to # TeamUSA and Canton Native @ JG...
4,b'Pleased to support @amergateways at their Ju...,"[(b'Pleased, VBN), (to, TO), (support, VB), (@...",b'Pleased to support @ amergateways at their J...


In [22]:
# Remove numbers, words <2 characters, punctuation, links and emojis 

def emoji_free_text(text): # From [9] 
    return emoji.get_emoji_regexp().sub(r'', text)

In [23]:
def clean_text(text: str) -> str:
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    cleaned_tokens = []
    for token in tokens:
        lower_token = token.lower()
        len_check = len(lower_token) >= 2
        #start_check = not (lower_token.startswith('http') or lower_token.startswith('\\') or lower_token.startswith(':'))
        stop_word_check = lower_token not in stop_words
        if len_check and stop_word_check:
            lower_token=re.sub(r'\d+', '', lower_token) 
            lower_token=re.sub(r'\b\w{1}\b', '',  lower_token)  
            lower_token=re.sub(r'[^\w\s]', '', lower_token)                 # Remove punctuation
            lower_token=re.sub(r'http\S+', '',lower_token)                 # Remove links
            cleaned_tokens.append(re.sub('[,.!?]|<br \/>\+|<br \/>', '', lower_token))
    return ' '.join(cleaned_tokens)


In [24]:
Text_clean = x['lemma_list'].map(clean_text)
Text_cleaner = Text_clean .map(emoji_free_text)
data_train['text_clean']=Text_cleaner

C:\Users\gokha\AppData\Local\Temp/ipykernel_66460/531451104.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(r'', text)


In [25]:
data_train.head()

,favorite_count,full_text,hashtags,retweet_count,year,party_id,party_class,text_clean
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,1,rt kusinews one longtime viewer congressman d...
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,1,today urge cdcgov immediately launch phone ...
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,1,tomorrow mo senior graduate calvary lutheran f...
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,1,congrats teamusa canton native jgreenway win w...
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,0,pleased support amergateways june fiesta honor...


In [27]:
# Cleaning test data set

# test datasindaki naleri de silecek miyiz?

# Tokenize
tokens_list_test=[word_tokenize(sent) for sent in data_test['full_text']]

In [28]:
# Pos Tag 
tagged_tokens_test =[pos_tag(sent) for sent in tokens_list_test]

In [29]:
x2=pd.DataFrame(data_test['full_text'])

In [30]:
x2['tagged_tokens_test']=tagged_tokens_test

In [32]:
lemma_list_test = x2['tagged_tokens_test'].transform(lambda value: ' '.join([lemmatizer.lemmatize(a[0],pos=get_wordnet_pos(a[1])) if get_wordnet_pos(a[1]) else a[0] for a in  value]))

In [96]:
x2['lemma_list_test']=lemma_list_test

In [98]:
Text_clean_test = x2['lemma_list_test'].map(clean_text)
Text_cleaner_test = Text_clean_test .map(emoji_free_text)
data_test['text_clean_test']=Text_cleaner_test

C:\Users\gokha\AppData\Local\Temp/ipykernel_66460/531451104.py:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(r'', text)


In [99]:
data_test

,Id,favorite_count,full_text,hashtags,retweet_count,year,party,party_class,text_clean_test
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D,0,taxreform improve playing field american worke...
1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D,0,this nativewomensequalpay day recommit pass pa...
2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D,0,xexci become convinced generation silence mak...
3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D,0,during nationaladoptionmonth honor adoptive pa...
4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D,0,happy airborneday usarmy paratrooper veteran s...
...,...,...,...,...,...,...,...,...,...
264995,264995,516,b'We need to #ExtendCHIP before a single child...,ExtendCHIP,223,2017.0,D,0,we need extendchip single child lose health co...
264996,264996,0,"b""Our #ObamaCare investigation continued today...",ObamaCare,3,2013.0,D,0,obamacare investigation continue today demand...
264997,264997,1,b'Congratulations to the new #MissTeenUSA Loga...,MissTeenUSA CT,4,2012.0,D,0,congratulations new missteenusa logan west sou...
264998,264998,2174,b'Speaking of dishonesty. Nothing like being c...,mosen,1168,2018.0,D,0,speaking dishonesty nothing like catch camera ...


BERT

In [112]:
X_train = data_train['text_clean']
y_train = data_train['party_class']

X_test = data_test['text_clean_test']
y_test = data_test['party_class']

In [37]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [38]:
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [39]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [40]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [41]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [42]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
18526/18526 [==============================] - 54173s 3s/step - loss: 0.6502 - accuracy: 0.6172
Epoch 2/5
18526/18526 [==============================] - 54230s 3s/step - loss: 0.6441 - accuracy: 0.6260
Epoch 3/5
18526/18526 [==============================] - 54225s 3s/step - loss: 0.6431 - accuracy: 0.6271
Epoch 4/5
18526/18526 [==============================] - 54244s 3s/step - loss: 0.6441 - accuracy: 0.6268
Epoch 5/5
18526/18526 [==============================] - 56731s 3s/step - loss: 0.6436 - accuracy: 0.6263


In [113]:
X_test

0         taxreform improve playing field american worke...
1         this nativewomensequalpay day recommit pass pa...
2          xexci become convinced generation silence mak...
3         during nationaladoptionmonth honor adoptive pa...
4         happy airborneday usarmy paratrooper veteran s...
                                ...                        
264995    we need extendchip single child lose health co...
264996     obamacare investigation continue today demand...
264997    congratulations new missteenusa logan west sou...
264998    speaking dishonesty nothing like catch camera ...
264999    in honor constitutionday remember foundation c...
Name: text_clean_test, Length: 265000, dtype: object

In [ ]:
y_test=model.predict(X_test)

In [52]:
y_test

array([[0.7372063 ],
       [0.44800308],
       [0.6934584 ],
       ...,
       [0.55734605],
       [0.6765712 ],
       [0.64794195]], dtype=float32)

In [60]:
y_test2 = y_test.flatten()

In [63]:
y_test2=pd.DataFrame(data=y_test, columns=['party'])

In [64]:
y_test2

,party
0,0.737206
1,0.448003
2,0.693458
3,0.682571
4,0.386390
...,...
264995,0.261586
264996,0.343167
264997,0.557346
264998,0.676571


In [65]:
y_test2 = np.where(y_test> 0.5, 1, 0)

In [66]:
y_test2

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [68]:
y_test2=pd.DataFrame(data=y_test2, columns=['party'])

In [69]:
y_test2

,party
0,1
1,0
2,1
3,1
4,0
...,...
264995,0
264996,0
264997,1
264998,1


In [70]:
y_test2.to_csv('ytest2_bert_istanbul.csv')

In [ ]:
y_test2.head()

In [71]:
y_test2['party']=y_test2['party'].apply(lambda x: 'D' if x==0 else 'R')

In [72]:
y_test2.to_csv('ytest2_bert_withparties_istanbul.csv')

In [73]:
y_test2.head()

,party
0,R
1,D
2,R
3,R
4,D
